In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Investigate break-up momentum 
Implement phasespace factor with separated square root for a cleaner cut structure:
$$
\begin{aligned}
\rho_\alpha(s) &= \frac{\sqrt{s-(m_{1,\alpha}-m_{2,\alpha})^2}\sqrt{s-(m_{1,\alpha}+m_{2,\alpha})^2}}{s} \\
q_\alpha(s) &= \frac{\sqrt{s-(m_{1,\alpha}-m_{2,\alpha})^2}\sqrt{s-(m_{1,\alpha}+m_{2,\alpha})^2}}{2\sqrt{s}}
\end{aligned}
$$

In [ ]:
from __future__ import annotations

from typing import TYPE_CHECKING, Any

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from IPython.display import Math

from ampform.dynamics.phasespace import (
    BreakupMomentum,
    BreakupMomentumSquared,
    PhaseSpaceFactor,
    _indices_to_subscript,  # noqa: PLC2701
)
from ampform.io import aslatex
from ampform.sympy import argument, determine_indices, unevaluated

if TYPE_CHECKING:
    from sympy.printing.latex import LatexPrinter

## Old definition of break-up momentum and phasespace factor

In [ ]:
@unevaluated
class PhaseSpaceFactorOld(sp.Expr):
    r"""Standard phase-space factor, using :func:`BreakupMomentumSquared`.

    See :pdg-review:`2021; Resonances; p.6`, Equation (50.9). We ignore the factor
    :math:`\frac{1}{16\pi}` as done in :cite:`chungPrimerKmatrixFormalism1995`, p.5.
    Note that this definition was replaced.
    """

    s: Any
    m1: Any
    m2: Any
    name: str | None = argument(default=None, sympify=False)

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q_squared = BreakupMomentumSquared(s, m1, m2)
        return 2 * sp.sqrt(q_squared) / sp.sqrt(s)

    def _latex_repr_(self, printer: LatexPrinter, *args) -> str:
        s_symbol = self.args[0]
        s_latex = printer._print(s_symbol)
        subscript = _indices_to_subscript(determine_indices(s_symbol))
        name = R"\rho" + subscript if self.name is None else self.name
        return Rf"{name}\left({s_latex}\right)"

In [ ]:
s, m1, m2 = sp.symbols("s m1 m2")
parameters = {m1: 0.2, m2: 0.6}

### New definition of break-up momentum and phasespace factor

In [ ]:
Math(aslatex({"q": BreakupMomentum(s, m1, m2).doit()}))

In [ ]:
Math(aslatex({"ρ": PhaseSpaceFactor(s, m1, m2).doit()}))  # noqa: RUF001

### Old definition of break-up momentum and phasespace factor

In [ ]:
Math(aslatex({"q": sp.sqrt(BreakupMomentumSquared(s, m1, m2)).doit()}))

In [ ]:
Math(aslatex({"ρ": PhaseSpaceFactorOld(s, m1, m2).doit()}))  # noqa: RUF001

## Plot phasespace factor and break-up momentum in the complex plane

In [ ]:
x_min, x_max = 0, +1.5
y_max = 0.7
z_max = 0.3
X, Y = np.meshgrid(
    np.linspace(x_min, x_max, num=500),
    np.linspace(-y_max, +y_max, num=300),
)
S = X + 1j * Y
ϵi = 1e-7j
thr_neg = (parameters[m1] - parameters[m2]) ** 2
thr_pos = (parameters[m1] + parameters[m2]) ** 2

rho_func = sp.lambdify(s, sp.I * PhaseSpaceFactor(s, m1, m2).doit().subs(parameters))
rho_old_func = sp.lambdify(
    s, sp.I * PhaseSpaceFactorOld(s, m1, m2).doit().subs(parameters)
)
q_func = sp.lambdify(s, BreakupMomentum(s, m1, m2).doit().subs(parameters))
q_old_func = sp.lambdify(
    s, sp.sqrt(BreakupMomentumSquared(s, m1, m2)).doit().subs(parameters)
)

fig, axes = plt.subplots(dpi=300, figsize=(7.5, 5), ncols=2, nrows=2, sharey=True)
fig.subplots_adjust(bottom=0, left=0, right=1, top=1, wspace=0.12)
ax1, ax2, ax3, ax4 = axes.flatten()
fig.patch.set_facecolor("none")
for ax in axes.flatten():
    ax.axvline((parameters[m1] + parameters[m2]) ** 2, ls="--", c="gray", lw=0.8)
    ax.patch.set_facecolor("none")
    ax.spines["bottom"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.set_xlabel(R"$\mathrm{Re}\,s$", labelpad=-12)
    ax.set_xticks([0])
ax1.set_ylabel(R"$\mathrm{Im}\,s$")
ax1.set_ylim(-y_max, +y_max)
ax1.set_yticks([0])

ax3.set_ylabel(R"$\mathrm{Im}\,s$")
ax3.set_ylim(-y_max, +y_max)
ax3.set_yticks([0])
style = dict(
    cmap="coolwarm",
    rasterized=True,
    vmin=-z_max,
    vmax=+z_max,
    zorder=-10,
)
mesh = ax1.pcolormesh(X, Y, rho_func(S).real, **style)
cbar = fig.colorbar(mesh, ax=ax1, pad=0.01)
cbar.ax.set_ylabel(R"$\mathrm{Re}\,i\rho$", labelpad=0, rotation=270)
cbar.ax.set_yticks([-z_max, +z_max])
cbar.ax.set_yticklabels(["$-$", "$+$"])
mesh = ax2.pcolormesh(X, Y, q_func(S).imag, **style)
cbar = fig.colorbar(mesh, ax=ax2, pad=0.01)
cbar.ax.set_ylabel(R"$\mathrm{Im}\,q$", labelpad=0, rotation=270)
cbar.ax.set_yticks([-z_max, +z_max])
cbar.ax.set_yticklabels(["$-$", "$+$"])
mesh = ax3.pcolormesh(X, Y, rho_old_func(S).real, **style)
cbar = fig.colorbar(mesh, ax=ax3, pad=0.01)
cbar.ax.set_ylabel(R"$\mathrm{Re}\,i\rho$", labelpad=0, rotation=270)
cbar.ax.set_yticks([-z_max, +z_max])
cbar.ax.set_yticklabels(["$-$", "$+$"])
mesh = ax4.pcolormesh(X, Y, q_old_func(S).imag, **style)
cbar = fig.colorbar(mesh, ax=ax4, pad=0.01)
cbar.ax.set_ylabel(R"$\mathrm{Im}\,q$", labelpad=0, rotation=270)
cbar.ax.set_yticks([-z_max, +z_max])
cbar.ax.set_yticklabels(["$-$", "$+$"])
ax1.plot(X[0], rho_func(X[0] + ϵi).real, c="#17365c", lw=0.5, zorder=-5)
ax1.plot(X[0], rho_func(X[0] + ϵi).imag, c="#8dae10", lw=0.5, zorder=-5)
ax2.plot(X[0], q_func(X[0] + ϵi).real, c="#17365c", lw=0.5, zorder=-5)
ax2.plot(X[0], q_func(X[0] + ϵi).imag, c="#8dae10", lw=0.5, zorder=-5)
ax3.plot(X[0], rho_old_func(X[0] + ϵi).real, c="#17365c", lw=0.5, zorder=-5)
ax3.plot(X[0], rho_old_func(X[0] + ϵi).imag, c="#8dae10", lw=0.5, zorder=-5)
ax4.plot(X[0], q_old_func(X[0] + ϵi).real, c="#17365c", lw=0.5, zorder=-5)
ax4.plot(X[0], q_old_func(X[0] + ϵi).imag, c="#8dae10", lw=0.5, zorder=-5)
ax1.text(
    0.99,
    0.47,
    R"$\mathrm{Re}\,i\rho$",
    c="#17365c",
    transform=ax1.transAxes,
    ha="right",
    va="top",
)
ax1.text(
    0.99,
    0.82,
    R"$\mathrm{Im}\,i\rho$",
    c="#8dae10",
    transform=ax1.transAxes,
    ha="right",
    va="top",
)
ax2.text(
    0.99,
    0.30,
    R"$\mathrm{Re}\,q$",
    c="#17365c",
    transform=ax2.transAxes,
    ha="right",
    va="top",
)
ax2.text(
    0.99,
    0.7,
    R"$\mathrm{Im}\, q$",
    c="#8dae10",
    transform=ax2.transAxes,
    ha="right",
    va="top",
)
ax3.text(
    0.99,
    0.47,
    R"$\mathrm{Re}\,i\rho$",
    c="#17365c",
    transform=ax3.transAxes,
    ha="right",
    va="top",
)
ax3.text(
    0.99,
    0.82,
    R"$\mathrm{Im}\,i\rho$",
    c="#8dae10",
    transform=ax3.transAxes,
    ha="right",
    va="top",
)
ax4.text(
    0.99,
    0.30,
    R"$\mathrm{Re}\,q$",
    c="#17365c",
    transform=ax4.transAxes,
    ha="right",
    va="top",
)
ax4.text(
    0.99,
    0.7,
    R"$\mathrm{Im}\, q$",
    c="#8dae10",
    transform=ax4.transAxes,
    ha="right",
    va="top",
)

ax1.hlines(0, thr_neg, thr_pos, color="black", lw=1.5)
ax1.scatter([thr_neg, thr_pos], [0, 0], color="black", s=25)
ax2.hlines(0, thr_pos, thr_neg, color="black", lw=1.5)
ax2.scatter([thr_neg, thr_pos], [0, 0], color="black", s=25)
ax1.set_title("Phasespace factor double sqrt", fontsize=10)
ax2.set_title("Break-up momentum factor double sqrt", fontsize=10)
ax3.set_title("Phasespace factor single sqrt", fontsize=10)
ax4.set_title("Break-up momentum factor single sqrt", fontsize=10)
plt.show(fig)

> [!NOTE]
> When defining the break-up momentum and the phasespace factor with a single square root in the numerator a more complex cut structure emerges when continuing the functions into the complex $s$-plane. 